In [6]:
import os
import collections
import requests
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support.ui import Select
import csv
import boto3

In [2]:
def set_up_parkrun_dir():
    if not os.path.exists('./parkrun'):
        os.mkdir('./parkrun')

def scrape_personal_results(driver, athelete_number):
    
    column_headings = [
        'event',
        'event_url',
        'run_date',
        'run_date_url',
        'run_number',
        'run_number_url',
        'pos',
        'time',
        'age_grade',
        'pb'
    ]
    
    set_up_parkrun_dir()
    
    previous_event_urls = []
    try:
        with open('./parkrun/personal_results.csv', 'r') as personal_result:
            race_result_reader = csv.DictReader(personal_result, fieldnames=column_headings)
            for row in race_result_reader:
                previous_event_urls.append(row['run_number_url'])
    except FileNotFoundError:
        pass
        
    all_results_url = 'https://www.parkrun.org.uk/results/athleteeventresultshistory/?athleteNumber={}&eventNumber=0'
    driver.get(all_results_url.format(athelete_number))
    run_rows = driver.find_elements_by_xpath('/html/body/div[1]/div[3]/div[1]/div/div[3]/table/tbody/tr[*]')
    with open('./parkrun/personal_results.csv', 'a+') as personal_results:  
        results_writer = csv.writer(personal_results)
        new_results = collections.deque()
        for row in run_rows:            
            run_number_url = row.find_element_by_xpath('td[3]/a').get_attribute('href')            
            if run_number_url in previous_event_urls:
                continue            
            data = [
                row.find_element_by_xpath('td[1]/a').text,
                row.find_element_by_xpath('td[1]/a').get_attribute('href'),
                row.find_element_by_xpath('td[2]/a').text,
                row.find_element_by_xpath('td[2]/a').get_attribute('href'),
                row.find_element_by_xpath('td[3]/a').text,
                run_number_url,
                row.find_element_by_xpath('td[4]').text,
                row.find_element_by_xpath('td[5]').text,
                row.find_element_by_xpath('td[6]').text,
                row.find_element_by_xpath('td[7]').text,
            ]
            new_results.appendleft(data)
        for data in new_results:
            results_writer.writerow(data)

In [4]:
driver = webdriver.Firefox()
scrape_personal_results(driver,'1839227')

In [3]:
def set_up_parkrun_race_results_dir():
    if not os.path.exists('./parkrun/race_results'):
        os.mkdir('./parkrun/race_results')
        
def close_search_tip_modal(driver):
    try:
        results_modal = driver.find_element_by_class_name('Results-modal-close')
        if results_modal:
            results_modal.click()
    except ElementNotInteractableException:
        pass
    
def scrape_results_for_event(event_name, event_number, results_url):
    set_up_parkrun_race_results_dir()
    race_result_filename = '{}_{}.csv'.format(event_name.lower().replace(' ', '_'), event_number)
    with open('./parkrun/race_results/{}'.format(race_result_filename), 'w+') as race_result:
        race_result_writer = csv.writer(race_result)
        driver.get(results_url)
        close_search_tip_modal(driver)
        select = Select(driver.find_element_by_name('display'))
        select.select_by_visible_text('Detailed')
        rows = driver.find_elements_by_xpath('/html/body/div[1]/div[3]/div[1]/div/div[1]/table/tbody/tr[*]')
        for row in rows:
            data = []
            # position
            data.append(row.find_element_by_xpath('td[1]').text)
            # Name
            try:
                data.append(row.find_element_by_xpath('td[2]/div').text)
            except NoSuchElementException:
                data.append('')
            # Name detail
            try:
                data.append(row.find_element_by_xpath('td[2]/div[2]').text)
            except NoSuchElementException:
                data.append('')
            # Gender
            try:
                data.append(row.find_element_by_xpath('td[3]/div').text)
            except NoSuchElementException:
                data.append('')
            # Gender detail
            try:
                data.append(row.find_element_by_xpath('td[3]/div[2]').text)
            except NoSuchElementException:
                data.append('')
            # Age Group
            try:
                data.append(row.find_element_by_xpath('td[4]/div').text)
            except NoSuchElementException:
                data.append('')
            # Age Group detail
            try:
                data.append(row.find_element_by_xpath('td[4]/div[2]').text)
            except NoSuchElementException:
                data.append('')
            # Club
            try:
                data.append(row.find_element_by_xpath('td[5]/div').text)  
            except NoSuchElementException:
                data.append('')
            # Time
            try: 
                data.append(row.find_element_by_xpath('td[6]/div[1]').text)
            except NoSuchElementException:
                data.append('')
            # Time detail
            try: 
                data.append(row.find_element_by_xpath('td[6]/div[2]').text)
            except NoSuchElementException:
                data.append('')  
            race_result_writer.writerow(data)
                    
def scrape_new_race_results(driver):
    with open('./parkrun/personal_results.csv', 'r') as personal_results:
        personal_results_reader = csv.reader(personal_results)
        for result in personal_results_reader:
            race_result_filename = '{}_{}.csv'.format(result[0].lower().replace(' ', '_'), result[4])
            if not os.path.exists('./parkrun/race_results/{}'.format(race_result_filename)):
                scrape_results_for_event(result[0], result[4], result[3])
    
def scrape_all_race_results(driver):
    with open('./parkrun/personal_results.csv', 'r') as personal_results:
        personal_results_reader = csv.reader(personal_results)
        for result in personal_results_reader:
            scrape_results_for_event(result[0], result[4], result[3])
            
        

In [5]:
scrape_new_race_results(driver)

In [21]:
session = boto3.Session(profile_name='personal')
dynamodb_client = session.client('dynamodb')

In [56]:
def update_persresults_database(client):
    with open('./parkrun/personal_results.csv', 'r') as personal_results_file:
        personal_results_reader = csv.reader(personal_results_file)
        for row in personal_results_reader:
            response = dynamodb_client.put_item(
                TableName = 'personal-results',
                Item = {
                    'date' : {
                        'S': row[2]
                    },
                    'event-name': {
                        'S': row[0]
                    },
                    'event_url': {
                        'S': row[1]
                    },
                    'run_date_url': {
                        'S': row[3]
                    },
                    'run_number': {
                        'N': row[4]
                    },
                    'run_number_url': {
                        'S': row[5]
                    },
                    'pos': {
                        'N': row[6]
                    },
                    'time': {
                        'S': row[7]
                    },
                    'age_grade': {
                        'N': row[8].strip('%')
                    },
                    'pb': {
                        'BOOL': True if row[9].strip() == 'PB' else False
                    }               

                }

            )
        
        

In [57]:
update_personal_results_database(dynamodb_client)

In [117]:
def update_race_results_database(client):
    race_results_filenames = {}
    with open('./parkrun/personal_results.csv', 'r') as personal_results_file:
        personal_results_reader = csv.reader(personal_results_file)
        for row in personal_results_reader:
            event_name = row[0].lower().replace(' ', '_')
            event_number= row[4]
            race_result_filename = '{}_{}.csv'.format(event_name, event_number)
            personal_results_reader = csv.reader(personal_results_file)
            race_results_filenames[race_result_filename] = {'event_name': event_name, 'event_number': event_number}
        
    for race_result_filename in race_results_filenames:
        with open('./parkrun/race_results/{}'.format(race_result_filename), 'r') as race_result:
            race_result_reader = csv.reader(race_result)
            row_num = 0
            for row in race_result_reader:
                print("Processing row {} of file {}".format(row_num, race_result_filename))
                row_num += 1
                
                item_mapping = {   
                    'event_name_number': {'type': 'S', 'data':'{}_{}'.format(race_results_filenames[race_result_filename]['event_name'], race_results_filenames[race_result_filename]['event_number'])},
                    'position': {'type': 'N', 'data':row[0]},
                    'name': {'type': 'S', 'data':row[1]},
                    'name_detail': {'type': 'S', 'data':row[2]},
                    'gender': {'type': 'S', 'data':row[3]},
                    'gender_detail': {'type': 'S', 'data':row[4]},
                    'age_group': {'type': 'S', 'data':row[5]},
                    'age_group_detail': {'type': 'S', 'data':row[6]},
                    'club': {'type': 'S', 'data':row[7]},
                    'time': {'type': 'S', 'data':row[8]},
                    'time_detail': {'type': 'S', 'data':row[9]},
                }
                
                item = {}
                
                for mapping in item_mapping:
                    if item_mapping[mapping]['data'] != '':
                        item[mapping] = {item_mapping[mapping]['type']: item_mapping[mapping]['data']}
                
                response = client.put_item(
                    TableName = 'race-results',
                    Item = item
                )
                
                
                
                



In [118]:
update_race_results_database(dynamodb_client)

Processing row 0 of file lincoln_86.csv
Processing row 1 of file lincoln_86.csv
Processing row 2 of file lincoln_86.csv
Processing row 3 of file lincoln_86.csv
Processing row 4 of file lincoln_86.csv
Processing row 5 of file lincoln_86.csv
Processing row 6 of file lincoln_86.csv
Processing row 7 of file lincoln_86.csv
Processing row 8 of file lincoln_86.csv
Processing row 9 of file lincoln_86.csv
Processing row 10 of file lincoln_86.csv
Processing row 11 of file lincoln_86.csv
Processing row 12 of file lincoln_86.csv
Processing row 13 of file lincoln_86.csv
Processing row 14 of file lincoln_86.csv
Processing row 15 of file lincoln_86.csv
Processing row 16 of file lincoln_86.csv
Processing row 17 of file lincoln_86.csv
Processing row 18 of file lincoln_86.csv
Processing row 19 of file lincoln_86.csv
Processing row 20 of file lincoln_86.csv
Processing row 21 of file lincoln_86.csv
Processing row 22 of file lincoln_86.csv
Processing row 23 of file lincoln_86.csv
Processing row 24 of file 

Processing row 198 of file lincoln_86.csv
Processing row 199 of file lincoln_86.csv
Processing row 200 of file lincoln_86.csv
Processing row 201 of file lincoln_86.csv
Processing row 202 of file lincoln_86.csv
Processing row 203 of file lincoln_86.csv
Processing row 204 of file lincoln_86.csv
Processing row 205 of file lincoln_86.csv
Processing row 206 of file lincoln_86.csv
Processing row 207 of file lincoln_86.csv
Processing row 208 of file lincoln_86.csv
Processing row 209 of file lincoln_86.csv
Processing row 210 of file lincoln_86.csv
Processing row 211 of file lincoln_86.csv
Processing row 212 of file lincoln_86.csv
Processing row 213 of file lincoln_86.csv
Processing row 214 of file lincoln_86.csv
Processing row 215 of file lincoln_86.csv
Processing row 216 of file lincoln_86.csv
Processing row 217 of file lincoln_86.csv
Processing row 218 of file lincoln_86.csv
Processing row 219 of file lincoln_86.csv
Processing row 220 of file lincoln_86.csv
Processing row 221 of file lincoln

Processing row 149 of file lincoln_87.csv
Processing row 150 of file lincoln_87.csv
Processing row 151 of file lincoln_87.csv
Processing row 152 of file lincoln_87.csv
Processing row 153 of file lincoln_87.csv
Processing row 154 of file lincoln_87.csv
Processing row 155 of file lincoln_87.csv
Processing row 156 of file lincoln_87.csv
Processing row 157 of file lincoln_87.csv
Processing row 158 of file lincoln_87.csv
Processing row 159 of file lincoln_87.csv
Processing row 160 of file lincoln_87.csv
Processing row 161 of file lincoln_87.csv
Processing row 162 of file lincoln_87.csv
Processing row 163 of file lincoln_87.csv
Processing row 164 of file lincoln_87.csv
Processing row 165 of file lincoln_87.csv
Processing row 166 of file lincoln_87.csv
Processing row 167 of file lincoln_87.csv
Processing row 168 of file lincoln_87.csv
Processing row 169 of file lincoln_87.csv
Processing row 170 of file lincoln_87.csv
Processing row 171 of file lincoln_87.csv
Processing row 172 of file lincoln

Processing row 93 of file lincoln_88.csv
Processing row 94 of file lincoln_88.csv
Processing row 95 of file lincoln_88.csv
Processing row 96 of file lincoln_88.csv
Processing row 97 of file lincoln_88.csv
Processing row 98 of file lincoln_88.csv
Processing row 99 of file lincoln_88.csv
Processing row 100 of file lincoln_88.csv
Processing row 101 of file lincoln_88.csv
Processing row 102 of file lincoln_88.csv
Processing row 103 of file lincoln_88.csv
Processing row 104 of file lincoln_88.csv
Processing row 105 of file lincoln_88.csv
Processing row 106 of file lincoln_88.csv
Processing row 107 of file lincoln_88.csv
Processing row 108 of file lincoln_88.csv
Processing row 109 of file lincoln_88.csv
Processing row 110 of file lincoln_88.csv
Processing row 111 of file lincoln_88.csv
Processing row 112 of file lincoln_88.csv
Processing row 113 of file lincoln_88.csv
Processing row 114 of file lincoln_88.csv
Processing row 115 of file lincoln_88.csv
Processing row 116 of file lincoln_88.csv

Processing row 6 of file lincoln_89.csv
Processing row 7 of file lincoln_89.csv
Processing row 8 of file lincoln_89.csv
Processing row 9 of file lincoln_89.csv
Processing row 10 of file lincoln_89.csv
Processing row 11 of file lincoln_89.csv
Processing row 12 of file lincoln_89.csv
Processing row 13 of file lincoln_89.csv
Processing row 14 of file lincoln_89.csv
Processing row 15 of file lincoln_89.csv
Processing row 16 of file lincoln_89.csv
Processing row 17 of file lincoln_89.csv
Processing row 18 of file lincoln_89.csv
Processing row 19 of file lincoln_89.csv
Processing row 20 of file lincoln_89.csv
Processing row 21 of file lincoln_89.csv
Processing row 22 of file lincoln_89.csv
Processing row 23 of file lincoln_89.csv
Processing row 24 of file lincoln_89.csv
Processing row 25 of file lincoln_89.csv
Processing row 26 of file lincoln_89.csv
Processing row 27 of file lincoln_89.csv
Processing row 28 of file lincoln_89.csv
Processing row 29 of file lincoln_89.csv
Processing row 30 of

...

Processing row 124 of file doddington_hall_3.csv
Processing row 125 of file doddington_hall_3.csv
Processing row 126 of file doddington_hall_3.csv
Processing row 127 of file doddington_hall_3.csv
Processing row 128 of file doddington_hall_3.csv
Processing row 129 of file doddington_hall_3.csv
Processing row 130 of file doddington_hall_3.csv
Processing row 131 of file doddington_hall_3.csv
Processing row 132 of file doddington_hall_3.csv
Processing row 133 of file doddington_hall_3.csv
Processing row 134 of file doddington_hall_3.csv
Processing row 135 of file doddington_hall_3.csv
Processing row 136 of file doddington_hall_3.csv
Processing row 137 of file doddington_hall_3.csv
Processing row 138 of file doddington_hall_3.csv
Processing row 139 of file doddington_hall_3.csv
Processing row 140 of file doddington_hall_3.csv
Processing row 141 of file doddington_hall_3.csv
Processing row 142 of file doddington_hall_3.csv
Processing row 143 of file doddington_hall_3.csv
Processing row 144 o

Processing row 292 of file doddington_hall_3.csv
Processing row 293 of file doddington_hall_3.csv
Processing row 294 of file doddington_hall_3.csv
Processing row 295 of file doddington_hall_3.csv
Processing row 296 of file doddington_hall_3.csv
Processing row 297 of file doddington_hall_3.csv
Processing row 298 of file doddington_hall_3.csv
Processing row 299 of file doddington_hall_3.csv
Processing row 300 of file doddington_hall_3.csv
Processing row 301 of file doddington_hall_3.csv
Processing row 302 of file doddington_hall_3.csv
Processing row 303 of file doddington_hall_3.csv
Processing row 304 of file doddington_hall_3.csv
Processing row 305 of file doddington_hall_3.csv
Processing row 306 of file doddington_hall_3.csv
Processing row 307 of file doddington_hall_3.csv
Processing row 308 of file doddington_hall_3.csv
Processing row 309 of file doddington_hall_3.csv
Processing row 310 of file doddington_hall_3.csv
Processing row 311 of file doddington_hall_3.csv
Processing row 312 o

Processing row 142 of file doddington_hall_4.csv
Processing row 143 of file doddington_hall_4.csv
Processing row 144 of file doddington_hall_4.csv
Processing row 145 of file doddington_hall_4.csv
Processing row 146 of file doddington_hall_4.csv
Processing row 147 of file doddington_hall_4.csv
Processing row 148 of file doddington_hall_4.csv
Processing row 149 of file doddington_hall_4.csv
Processing row 150 of file doddington_hall_4.csv
Processing row 151 of file doddington_hall_4.csv
Processing row 152 of file doddington_hall_4.csv
Processing row 153 of file doddington_hall_4.csv
Processing row 154 of file doddington_hall_4.csv
Processing row 155 of file doddington_hall_4.csv
Processing row 156 of file doddington_hall_4.csv
Processing row 157 of file doddington_hall_4.csv
Processing row 158 of file doddington_hall_4.csv
Processing row 159 of file doddington_hall_4.csv
Processing row 160 of file doddington_hall_4.csv
Processing row 161 of file doddington_hall_4.csv
Processing row 162 o

Processing row 44 of file doddington_hall_5.csv
Processing row 45 of file doddington_hall_5.csv
Processing row 46 of file doddington_hall_5.csv
Processing row 47 of file doddington_hall_5.csv
Processing row 48 of file doddington_hall_5.csv
Processing row 49 of file doddington_hall_5.csv
Processing row 50 of file doddington_hall_5.csv
Processing row 51 of file doddington_hall_5.csv
Processing row 52 of file doddington_hall_5.csv
Processing row 53 of file doddington_hall_5.csv
Processing row 54 of file doddington_hall_5.csv
Processing row 55 of file doddington_hall_5.csv
Processing row 56 of file doddington_hall_5.csv
Processing row 57 of file doddington_hall_5.csv
Processing row 58 of file doddington_hall_5.csv
Processing row 59 of file doddington_hall_5.csv
Processing row 60 of file doddington_hall_5.csv
Processing row 61 of file doddington_hall_5.csv
Processing row 62 of file doddington_hall_5.csv
Processing row 63 of file doddington_hall_5.csv
Processing row 64 of file doddington_hal

Processing row 213 of file doddington_hall_5.csv
Processing row 214 of file doddington_hall_5.csv
Processing row 215 of file doddington_hall_5.csv
Processing row 216 of file doddington_hall_5.csv
Processing row 217 of file doddington_hall_5.csv
Processing row 218 of file doddington_hall_5.csv
Processing row 219 of file doddington_hall_5.csv
Processing row 220 of file doddington_hall_5.csv
Processing row 221 of file doddington_hall_5.csv
Processing row 222 of file doddington_hall_5.csv
Processing row 223 of file doddington_hall_5.csv
Processing row 224 of file doddington_hall_5.csv
Processing row 225 of file doddington_hall_5.csv
Processing row 226 of file doddington_hall_5.csv
Processing row 227 of file doddington_hall_5.csv
Processing row 228 of file doddington_hall_5.csv
Processing row 229 of file doddington_hall_5.csv
Processing row 230 of file doddington_hall_5.csv
Processing row 231 of file doddington_hall_5.csv
Processing row 232 of file doddington_hall_5.csv
Processing row 233 o